In [368]:
import pandas as pd
import yaml
from sqlalchemy import create_engine
from datetime import timedelta

## Database Connection

In [369]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [370]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [371]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

In [372]:
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas sin saltos
pd.set_option('display.width', 1000)  

## Extraction


In [373]:
fase_servicio = pd.read_sql_table('dim_fase_servicio', etl_conn) 

In [374]:
total_nan = fase_servicio.isna().sum()
print(total_nan)
print(fase_servicio.columns)

key_fase_servicio    0
fecha                0
hora                 0
estado_id            0
servicio_id          0
dtype: int64
Index(['key_fase_servicio', 'fecha', 'hora', 'estado_id', 'servicio_id'], dtype='object')


In [375]:
print(len(fase_servicio))

128402


## Transformation


In [376]:
# Crea un DataFrame único con las claves y atributos de interés
fase_unique = fase_servicio[["fecha", "hora", "estado_id", "servicio_id", "key_fase_servicio"]].drop_duplicates(subset=["fecha", "hora", "estado_id", "servicio_id"])

# Realizamos el merge sin renombrar las columnas, usando 'key_fase_servicio' para la unión
hecho_fase_servicio = pd.merge(fase_servicio, fase_unique[["key_fase_servicio", "servicio_id", "fecha", "hora", "estado_id"]],
                               on=["key_fase_servicio", "servicio_id", "fecha", "hora", "estado_id"], how="left")

# Verificar el resultado final
print(hecho_fase_servicio.head(20))


    key_fase_servicio      fecha      hora  estado_id  servicio_id
0                   0 2024-01-29  01:13:32          4          226
1                   1 2024-01-30  18:45:12          3           79
2                   2 2024-02-06  11:34:04          5          613
3                   3 2024-02-01  14:50:39          4          376
4                   4 2024-04-06  16:11:21          3         7164
5                   5 2024-02-04  11:15:40          3          377
6                   6 2024-05-03  06:11:39          3        10910
7                   7 2024-02-07  16:26:03          2          746
8                   8 2024-02-09  13:25:05          6          842
9                   9 2024-02-10  13:46:43          2          930
10                 10 2024-02-12  09:01:02          2          974
11                 11 2024-02-12  10:42:23          4          991
12                 12 2024-02-13  15:44:23          5         1093
13                 13 2024-02-14  08:42:30          4         

## calculo tiempo por fase_servicio

In [377]:
import pandas as pd

# Ordenar por 'key_fase_servicio' y 'fecha'
hecho_fase_servicio = fase_servicio.sort_values(by=['servicio_id', 'fecha'])

# Asegurarse de que la columna 'hora' esté en formato timedelta
hecho_fase_servicio['hora'] = pd.to_timedelta(hecho_fase_servicio['hora'].astype(str))

# Calcular la diferencia en días entre fases consecutivas dentro de cada servicio
hecho_fase_servicio['dias_de_demora'] = hecho_fase_servicio.groupby('servicio_id')['fecha'].transform(lambda x: x.diff().dt.days)

# Asignar el resultado a la columna directamente para evitar errores
hecho_fase_servicio['dias_de_demora'] = hecho_fase_servicio['dias_de_demora'].fillna(0)

# Calcular la diferencia en horas entre fases consecutivas dentro de cada servicio
hecho_fase_servicio['hora_de_demora'] = hecho_fase_servicio.groupby('servicio_id')['hora'].transform(lambda x: x.diff())

# Corregir las horas negativas: Si la hora actual es menor que la anterior, agregar 24 horas
hecho_fase_servicio['hora_de_demora'] = hecho_fase_servicio['hora_de_demora'].apply(lambda x: x if x >= pd.Timedelta(0) else x + pd.Timedelta(days=1))

# Convertir la diferencia en horas
hecho_fase_servicio['hora_de_demora'] = hecho_fase_servicio['hora_de_demora'].dt.total_seconds() / 3600

# Llenar valores nulos en 'hora_de_demora' con 0
hecho_fase_servicio['hora_de_demora'] = hecho_fase_servicio['hora_de_demora'].fillna(0)

# Convertir las columnas 'hora' y 'hora_de_demora' a su formato de horas
hecho_fase_servicio['hora'] = hecho_fase_servicio['hora'].dt.total_seconds() / 3600
hecho_fase_servicio['hora_de_demora'] = hecho_fase_servicio['hora_de_demora'].apply(lambda x: round(x, 2))  # Redondear a 2 decimales

# Mostrar el resultado final
hecho_fase_servicio.head(20)


,key_fase_servicio,fecha,hora,estado_id,servicio_id,dias_de_demora,hora_de_demora
301,301,2023-09-19,16.371667,1,7,0.0,0.00
667,667,2023-10-13,17.855556,2,7,24.0,1.48
686,686,2023-10-31,12.046901,4,7,18.0,18.19
687,687,2023-10-31,17.131944,5,7,0.0,5.09
688,688,2023-10-31,12.266667,6,7,0.0,19.13
302,302,2023-09-19,16.501389,1,8,0.0,0.00
792,792,2023-12-20,20.245278,2,8,92.0,3.74
5902,5902,2024-02-14,15.571667,4,8,56.0,19.33
34483,34483,2024-04-09,16.143056,5,8,55.0,0.57
304,304,2023-09-19,16.501389,1,9,0.0,0.00


## Calculo atributo tiempo promedio general

In [378]:
import pandas as pd

# Asegúrate de trabajar con 'hecho_fase_servicio' en lugar de 'fase_servicio'

# Convertir 'hora_de_demora' a timedelta si aún no está en el formato adecuado
hecho_fase_servicio['hora_de_demora'] = pd.to_timedelta(hecho_fase_servicio['hora_de_demora'], unit='h')

# Calcular el tiempo total en días considerando solo las fechas, con dos decimales
hecho_fase_servicio['tiempo_total'] = hecho_fase_servicio.groupby('servicio_id')['fecha'].transform(lambda x: (x.max() - x.min()).days).round(2)

# Calcular el tiempo total en horas sumando las horas de demora de cada fase por servicio
hecho_fase_servicio['tiempo_total_horas'] = hecho_fase_servicio.groupby('servicio_id')['hora_de_demora'].transform('sum').dt.total_seconds() / 3600
hecho_fase_servicio['tiempo_total_horas'] = hecho_fase_servicio['tiempo_total_horas'].round(2)

# Calcular el número de fases por servicio
hecho_fase_servicio['numero_fases'] = hecho_fase_servicio.groupby('servicio_id')['fecha'].transform('count')

# Calcular el tiempo promedio por fase en días (dividiendo el tiempo total de días entre el número de fases)
hecho_fase_servicio['tiempo_promedio_dias'] = (hecho_fase_servicio['tiempo_total'] / hecho_fase_servicio['numero_fases']).round(2)

# Calcular el tiempo promedio en horas por fase con dos decimales (dividiendo el tiempo total de horas entre el número de fases)
hecho_fase_servicio['tiempo_promedio_horas'] = (hecho_fase_servicio['tiempo_total_horas'] / hecho_fase_servicio['numero_fases']).round(2)

# Convertir la columna 'hora_de_demora' a su formato de horas (si aún no se ha hecho)
hecho_fase_servicio['hora_de_demora'] = hecho_fase_servicio['hora_de_demora'].dt.total_seconds() / 3600

# Mostrar los primeros 10 resultados
hecho_fase_servicio.head(10)


,key_fase_servicio,fecha,hora,estado_id,servicio_id,dias_de_demora,hora_de_demora,tiempo_total,tiempo_total_horas,numero_fases,tiempo_promedio_dias,tiempo_promedio_horas
301,301,2023-09-19,16.371667,1,7,0.0,0.00,42,43.89,5,8.40,8.78
667,667,2023-10-13,17.855556,2,7,24.0,1.48,42,43.89,5,8.40,8.78
686,686,2023-10-31,12.046901,4,7,18.0,18.19,42,43.89,5,8.40,8.78
687,687,2023-10-31,17.131944,5,7,0.0,5.09,42,43.89,5,8.40,8.78
688,688,2023-10-31,12.266667,6,7,0.0,19.13,42,43.89,5,8.40,8.78
302,302,2023-09-19,16.501389,1,8,0.0,0.00,203,23.64,4,50.75,5.91
792,792,2023-12-20,20.245278,2,8,92.0,3.74,203,23.64,4,50.75,5.91
5902,5902,2024-02-14,15.571667,4,8,56.0,19.33,203,23.64,4,50.75,5.91
34483,34483,2024-04-09,16.143056,5,8,55.0,0.57,203,23.64,4,50.75,5.91
304,304,2023-09-19,16.501389,1,9,0.0,0.00,100,3.05,2,50.00,1.52


## Eliminar filas

In [379]:
hecho_fase_servicio.drop(columns=["fecha","hora","estado_id","servicio_id"], inplace=True)
hecho_fase_servicio

,key_fase_servicio,dias_de_demora,hora_de_demora,tiempo_total,tiempo_total_horas,numero_fases,tiempo_promedio_dias,tiempo_promedio_horas
301,301,0.0,0.00,42,43.89,5,8.4,8.78
667,667,24.0,1.48,42,43.89,5,8.4,8.78
686,686,18.0,18.19,42,43.89,5,8.4,8.78
687,687,0.0,5.09,42,43.89,5,8.4,8.78
688,688,0.0,19.13,42,43.89,5,8.4,8.78
...,...,...,...,...,...,...,...,...
128400,128400,0.0,0.21,0,1.03,5,0.0,0.21
128388,128388,0.0,0.00,0,0.52,4,0.0,0.13
128390,128390,0.0,0.28,0,0.52,4,0.0,0.13
128393,128393,0.0,0.19,0,0.52,4,0.0,0.13


## Load

In [380]:
fase_servicio.to_sql("hecho_servicio_fase", etl_conn, if_exists="replace", index_label="key_servicio_fase") 

402